In [3]:
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
# author: Hari S. Muralidharan

In [4]:
def Qualify_Alignments(row, contain = 0.95, overhang_cutoff = 100):
    if ((row['Overhang'] > overhang_cutoff) and (row['Overhang']/row['AlignLength'] > 0.1) and 
        (row['AlignLength']/min(row['QLen'], row['SLen']) < contain)):
        return "Internal_Match"
    
    elif ((row['QStart'] <= row['SStart']) and  
          (row['AlignLength']/min(row['QLen'], row['SLen']) >= contain)):
        return "q in s"
    
    elif ((row['QStart'] >= row['SStart']) and 
          (row['AlignLength']/min(row['QLen'], row['SLen']) >= contain)):
        return "s in q"
    
    elif row['QStart'] > row['SStart']:
        return "q overlaps s"
    
    else:
        return "s overlaps q"

def Build_Graph(df_Containments):
    edge_list = list(zip(df_Containments['Query'].tolist(), df_Containments['Subject'].tolist()))
    G = nx.DiGraph(edge_list)
    nodes = df_Containments['Query'].tolist()+df_Containments['Subject'].tolist()
   
    d_length = dict(zip(nodes, df_Containments['QLen'].tolist()+df_Containments['SLen'].tolist()))
    nx.set_node_attributes(G, d_length, name="length")

    d_edgetype = dict(zip(edge_list, df_Containments['Alignment_Type'].tolist()))
    nx.set_edge_attributes(G, d_edgetype, name="type")

    d_qcov = dict(zip(edge_list, df_Containments['QCov'].tolist()))
    nx.set_edge_attributes(G, d_qcov, name="qcov")

    d_scov = dict(zip(edge_list, df_Containments['SCov'].tolist()))
    nx.set_edge_attributes(G, d_scov, name="scov")

    d_qstart = dict(zip(edge_list, df_Containments['QStart'].tolist()))
    nx.set_edge_attributes(G, d_qstart, name="qstart")

    d_qend = dict(zip(edge_list, df_Containments['QEnd'].tolist()))
    nx.set_edge_attributes(G, d_qend, name="qend")

    d_sstart = dict(zip(edge_list, df_Containments['SStart'].tolist()))
    nx.set_edge_attributes(G, d_sstart, name="sstart")

    d_send = dict(zip(edge_list, df_Containments['SEnd'].tolist()))
    nx.set_edge_attributes(G, d_send, name="send")
    
    return G

def Simplify_Containment_Clusters(g, cluster_id):
    containment_clusters = []
    d_lengths = nx.get_node_attributes(g, "length")
    nodes, lengths = np.array(list(d_lengths.keys())), np.array(list(d_lengths.values()))
    nodes_sorted = nodes[np.argsort(lengths)[::-1]]
    
    visited = set({})
    
    for n in nodes_sorted:
        if n not in visited:
            cluster = [n] + list(g.neighbors(n))
            op = []
            for c in cluster[1:]:
                d = {'RepresentativeContig':n, 'Contig':c, 'EdgeType':g.edges[(n,c)]['type'], 
                     'RepresentativeLength':g.nodes[n]['length'], 'ContigLength':g.nodes[c]['length'],
                     'qstart':g.edges[(n,c)]['qstart'], 'qend':g.edges[(n,c)]['qend'], 
                     'sstart':g.edges[(n,c)]['sstart'], 'send':g.edges[(n,c)]['send'],
                     'Cluster_ID':cluster_id}
                op.append(d)
            cluster_id += 1
            g.remove_nodes_from(cluster)
            visited.update(cluster)
            if len(cluster) > 1:
                containment_clusters+= op
    return containment_clusters

def Load_PAF(filepath):
    header = ['Query','QLen','QStart','QEnd','Orientation','Subject',
              'SLen','SStart','SEnd','Matches','AlignLength','MAPQ']
    op = []
    with open(filepath) as fileobject:
        for l in fileobject:
            l = l.split('\t')[:12]
            op.append(dict(zip(header, l)))
    df = pd.DataFrame(op)
    df[['QLen','QStart','QEnd','SLen','SStart',
        'SEnd','Matches','AlignLength','MAPQ']] = df[['QLen','QStart','QEnd','SLen','SStart',
                                                      'SEnd','Matches','AlignLength','MAPQ']].astype('int')
    df['PIdent'] = df['Matches']/df['AlignLength']*100
    df['QCov'] = df['AlignLength']/df['QLen']*100
    df['SCov'] = df['AlignLength']/df['SLen']*100
    df['Overhang'] = (np.minimum(df['SStart'],df['QStart']) + np.minimum(df['QLen']-df['QEnd'], 
                                                                         df['SLen']-df['SEnd']))
    return df

In [5]:
data_path = '/Users/harihara/Mount/Phage-Detection/Potential_Phages_Updated/Minimap2_Alignments.paf'
df = Load_PAF(data_path)
df = df[df['Query'] != df['Subject']]

In [6]:
len(set(df['Query'].tolist()) | set(df['Subject'].tolist())) 

77949

In [36]:
percent_identity_cutoff = 70
containment_cutoff = 0.80
overhang_cutoff = 100

In [37]:
df_filtered = df.loc[df['PIdent'] >= percent_identity_cutoff, :].copy()
df_filtered = df_filtered.loc[df_filtered.groupby(['Query','Subject'])['AlignLength'].idxmax()]
df_filtered.loc[:,'Alignment_Type'] = df_filtered.apply(partial(Qualify_Alignments, 
                                                                contain = containment_cutoff,
                                                                overhang_cutoff = overhang_cutoff), axis = 1)

In [38]:
df_Containments = df_filtered.loc[(df_filtered['Alignment_Type'] == 'q in s')|
                                  (df_filtered['Alignment_Type'] == 's in q'), :].copy()
G = Build_Graph(df_Containments)

In [42]:
conn = list(nx.weakly_connected_components(G))
containment_clusters = []
cluster_id = 0
for c in conn:
    g = nx.Graph(G.subgraph(c))
    clusters = Simplify_Containment_Clusters(g, cluster_id)
    if len(clusters) > 1: cluster_id = clusters[-1]['Cluster_ID']+1
    containment_clusters += clusters
df_containment_clusters = pd.DataFrame(containment_clusters)
df_containment_clusters.to_csv('Phage_Clusters.txt', sep = "\t")